In [1]:
# Les fonctions utiles ici
import pandas as pd
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
import torch
import time
from geometry import RectangleWithoutCylinder
import numpy as np

In [5]:
def find_x_branch(ya0, w0, nb_branch, t_min, t_max):
    time_interval = torch.linspace(t_min, t_max, nb_branch)
    return ya0 * (w0**2) * torch.cos(w0 * time_interval)

def charge_data(hyper_param, param_adim):
    """
    Charge the data of X_full, U_full with every points
    And X_train, U_train with less points
    """
    time_start_charge = time.time()
    nb_simu = len(hyper_param["file"])
    x_full, y_full, t_full, x_branch_full = [], [], [], []
    u_full, v_full, p_full = [], [], []
    x_norm_full, y_norm_full, t_norm_full, x_branch_norm_full = [], [], [], []
    u_norm_full, v_norm_full, p_norm_full = [], [], []

    f = 0.5 * (hyper_param["H"] / hyper_param["m"]) ** 0.5
    w0 = 2 * torch.pi * f
    time_tot = hyper_param['nb_period_plot'] / f  # la fréquence de l'écoulement
    t_max = hyper_param['t_min'] + hyper_param['nb_period'] * time_tot
    t_max_plot = hyper_param['t_min'] + time_tot
    for k in range(nb_simu):
        df = pd.read_csv("data/" + hyper_param["file"][k])
        df_modified = df.loc[
            (df["Points:0"] >= hyper_param["x_min"])
            & (df["Points:0"] <= hyper_param["x_max"])
            & (df["Points:1"] >= hyper_param["y_min"])
            & (df["Points:1"] <= hyper_param["y_max"])
            & (df["Time"] > hyper_param["t_min"])
            & (df["Time"] < t_max)
            & (df["Points:2"] == 0.0)
            & (df["Points:0"] ** 2 + df["Points:1"] ** 2 > (0.025 / 2) ** 2),
            :,
        ].copy()

        # Adimensionnement
        x_full.append(
            torch.tensor(df_modified["Points:0"].to_numpy(), dtype=torch.float32)
            / param_adim["L"]
        )
        y_full.append(
            torch.tensor(df_modified["Points:1"].to_numpy(), dtype=torch.float32)
            / param_adim["L"]
        )
        time_without_modulo = df_modified["Time"].to_numpy() - hyper_param['t_min']
        time_with_modulo = hyper_param['t_min'] + time_without_modulo % (1/f)
        t_full.append(
            torch.tensor(time_with_modulo, dtype=torch.float32)
            / (param_adim["L"] / param_adim["V"])
        )
        x_branch_full.append(
                find_x_branch(
                hyper_param['ya0'][k],
                w0=w0,
                nb_branch=hyper_param['nb_entry_branch'],
                t_min=hyper_param['t_min'],
                t_max=t_max_plot).reshape(-1,1).repeat(1, x_full[-1].shape[0]).T
        )

        u_full.append(
            torch.tensor(df_modified["Velocity:0"].to_numpy(), dtype=torch.float32)
            / param_adim["V"]
        )
        v_full.append(
            torch.tensor(df_modified["Velocity:1"].to_numpy(), dtype=torch.float32)
            / param_adim["V"]
        )
        p_full.append(
            torch.tensor(df_modified["Pressure"].to_numpy(), dtype=torch.float32)
            / ((param_adim["V"] ** 2) * param_adim["rho"])
        )
        print(f"fichier n°{k} chargé")

    # les valeurs pour renormaliser ou dénormaliser
    mean_std = {
        "u_mean": torch.cat([u for u in u_full], dim=0).mean(),
        "v_mean": torch.cat([v for v in v_full], dim=0).mean(),
        "p_mean": torch.cat([p for p in p_full], dim=0).mean(),
        "x_mean": torch.cat([x for x in x_full], dim=0).mean(),
        "y_mean": torch.cat([y for y in y_full], dim=0).mean(),
        "t_mean": torch.cat([t for t in t_full], dim=0).mean(),
        "x_std": torch.cat([x for x in x_full], dim=0).std(),
        "y_std": torch.cat([y for y in y_full], dim=0).std(),
        "t_std": torch.cat([t for t in t_full], dim=0).std(),
        "u_std": torch.cat([u for u in u_full], dim=0).std(),
        "v_std": torch.cat([v for v in v_full], dim=0).std(),
        "p_std": torch.cat([p for p in p_full], dim=0).std(),
        "x_branch_std": torch.cat([x_b for x_b in x_branch_full], dim=0).std(dim=0),
        "x_branch_mean": torch.cat([x_b for x_b in x_branch_full], dim=0).mean(dim=0)
    }
    X_trunk_full = torch.zeros((0, 3))
    U_full = torch.zeros((0, 3))

    for k in range(nb_simu):
        # Normalisation Z
        x_norm_full.append((x_full[k] - mean_std["x_mean"]) / mean_std["x_std"])
        y_norm_full.append((y_full[k] - mean_std["y_mean"]) / mean_std["y_std"])
        t_norm_full.append((t_full[k] - mean_std["t_mean"]) / mean_std["t_std"])
        p_norm_full.append((p_full[k] - mean_std["p_mean"]) / mean_std["p_std"])
        u_norm_full.append((u_full[k] - mean_std["u_mean"]) / mean_std["u_std"])
        v_norm_full.append((v_full[k] - mean_std["v_mean"]) / mean_std["v_std"])
        x_branch_norm_full.append((x_branch_full[k] - mean_std["x_branch_mean"]) / mean_std["x_branch_std"])
        X_trunk_full = torch.cat(
            (
                X_trunk_full,
                torch.stack(
                    (
                        x_norm_full[-1],
                        y_norm_full[-1],
                        t_norm_full[-1],
                    ),
                    dim=1,
                ),
            )
        )
        U_full = torch.cat(
            (
                U_full,
                torch.stack((u_norm_full[-1], v_norm_full[-1], p_norm_full[-1]), dim=1),
            )
        )

    X_branch_full = (torch.cat([x_b for x_b in x_branch_norm_full]))
    X_trunk_train = torch.zeros((0, 3))
    U_train = torch.zeros((0, 3))
    X_branch_train = torch.zeros((0, hyper_param['nb_entry_branch']))
    print("Starting X_train")
    for nb in range(len(x_full)):
        print(f"Simu n°{nb}/{len(t_norm_full)}")
        print(f"Time:{(time.time()-time_start_charge):.3f}")
        for time_ in torch.unique(t_norm_full[nb]):
            # les points autour du cylindre dans un rayon de hyper_param['rayon_proche']
            masque = (
                ((x_full[nb] ** 2 + y_full[nb] ** 2) < ((hyper_param['rayon_close_cylinder'] / param_adim["L"]) ** 2))
                & (t_norm_full[nb] == time_)
            )
            indices = torch.randperm(len(x_norm_full[nb][masque]))[
                : hyper_param["nb_points_close_cylinder"]
            ]

            new_x = torch.stack(
                (
                    x_norm_full[nb][masque][indices],
                    y_norm_full[nb][masque][indices],
                    t_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            new_y = torch.stack(
                (
                    u_norm_full[nb][masque][indices],
                    v_norm_full[nb][masque][indices],
                    p_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            X_trunk_train = torch.cat((X_trunk_train, new_x))
            U_train = torch.cat((U_train, new_y))
            X_branch_train = torch.cat((X_branch_train, x_branch_norm_full[nb][indices, :]), dim=0)


            # Les points avec 'latin hypercube sampling'
            time_start_lhs = time.time()

            masque = (t_norm_full[nb] == time_) 
            if x_norm_full[nb][masque].size(0) > 0:
                indices = torch.randperm(x_norm_full[nb][masque].size(0))[
                    : hyper_param["nb_points"]
                ]
                new_x = torch.stack(
                    (
                        x_norm_full[nb][masque][indices],
                        y_norm_full[nb][masque][indices],
                        t_norm_full[nb][masque][indices],
                    ),
                    dim=1,
                )
                new_y = torch.stack(
                    (
                        u_norm_full[nb][masque][indices],
                        v_norm_full[nb][masque][indices],
                        p_norm_full[nb][masque][indices],
                    ),
                    dim=1,
                )
                X_trunk_train = torch.cat((X_trunk_train, new_x))
                U_train = torch.cat((U_train, new_y))
                X_branch_train = torch.cat((X_branch_train, x_branch_norm_full[nb][indices, :]))
                indices = torch.randperm(X_trunk_train.size(0))

    # les points du bord
    teta_int = torch.linspace(0, 2 * torch.pi, hyper_param["nb_points_border"])
    X_trunk_border = torch.empty((0, 3))
    X_branch_border = torch.empty((0, hyper_param['nb_entry_branch']))
    x_ = (
        (((0.025 / 2) * torch.cos(teta_int)) / param_adim["L"]) - mean_std["x_mean"]
    ) / mean_std["x_std"]
    y_ = (
        (((0.025 / 2) * torch.sin(teta_int)) / param_adim["L"]) - mean_std["y_mean"]
    ) / mean_std["y_std"]

    for nb in range(len(x_branch_full)):
        for time_ in torch.unique(t_norm_full[nb]):
            new_x = torch.stack(
                (x_, y_, torch.ones_like(x_) * time_), dim=1
            )
            X_trunk_border = torch.cat((X_trunk_border, new_x))
            X_branch_border = torch.cat((X_branch_border, x_branch_norm_full[nb][0].reshape(-1, 1).T.repeat(x_.shape[0], 1)))
        indices = torch.randperm(X_trunk_border.size(0))
        X_trunk_border = X_trunk_border[indices]
        X_branch_border = X_branch_border[indices]
    print("X_border OK")

    teta_int_test = torch.linspace(0, 2 * torch.pi, 15)
    X_trunk_border_test = torch.zeros((0, 3))
    X_branch_border_test = torch.empty((0, hyper_param['nb_entry_branch']))
    x_ = (
        (((0.025 / 2) * torch.cos(teta_int_test)) / param_adim["L"])
        - mean_std["x_mean"]
    ) / mean_std["x_std"]
    y_ = (
        (((0.025 / 2) * torch.sin(teta_int_test)) / param_adim["L"])
        - mean_std["y_mean"]
    ) / mean_std["y_std"]

    for nb in range(len(x_branch_full)):
        for time_ in torch.unique(t_norm_full[nb]):
            new_x = torch.stack(
                (x_, y_, torch.ones_like(x_) * time_), dim=1
            )
            X_trunk_border_test = torch.cat((X_trunk_border_test, new_x))
            X_branch_border_test = torch.cat((X_branch_border_test, x_branch_norm_full[nb][0].reshape(-1, 1).T.repeat(x_.shape[0], 1)))

    points_coloc_test = np.random.choice(
        len(X_trunk_full), hyper_param["n_data_test"], replace=False
    )
    X_trunk_test_data = X_trunk_full[points_coloc_test]
    U_test_data = U_full[points_coloc_test]
    X_branch_test_data = X_branch_full[points_coloc_test]
    
    X_test_data = (X_branch_test_data, X_trunk_test_data)
    X_train = (X_branch_train, X_trunk_train)
    X_border = (X_branch_border, X_trunk_border)
    X_border_test = (X_branch_border_test, X_trunk_border_test)
    X_full = (X_branch_full, X_trunk_full)
    return X_train, U_train, X_full, U_full, X_border, X_border_test, mean_std, X_test_data, U_test_data


            

In [6]:
X_train, U_train, X_full, U_full, X_border, X_border_test, mean_std, X_test_data, U_test_data = charge_data(hyper_param, param_adim)

fichier n°0 chargé
fichier n°1 chargé
fichier n°2 chargé
fichier n°3 chargé
fichier n°4 chargé
Starting X_train
Simu n°0/5
Time:8.591
Simu n°1/5
Time:10.207
Simu n°2/5
Time:12.272
Simu n°3/5
Time:15.208
Simu n°4/5
Time:18.514
X_border OK


In [11]:
X_border_test

(tensor([[ 1.4837,  1.4837,  1.4837,  ...,  1.4836,  1.4836,  1.4837],
         [ 1.4837,  1.4837,  1.4837,  ...,  1.4836,  1.4836,  1.4837],
         [ 1.4837,  1.4837,  1.4837,  ...,  1.4836,  1.4836,  1.4837],
         ...,
         [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
         [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
         [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494]]),
 tensor([[ 2.6294e-01, -5.9921e-10, -1.7117e+00],
         [ 2.3115e-01,  1.9162e-01, -1.7117e+00],
         [ 1.4207e-01,  3.4530e-01, -1.7117e+00],
         ...,
         [ 1.4207e-01, -3.4530e-01,  1.7208e+00],
         [ 2.3115e-01, -1.9162e-01,  1.7208e+00],
         [ 2.6294e-01,  7.6621e-08,  1.7208e+00]]))

In [3]:
hyper_param = {
    "H": 230.67,  # la rigidité du ressort
    "ya0": [
        0.00125,
        0.00375,
        0.00625,
        0.00875,
        0.01,
    ],  # la position initiale du ressort
    "m": 1.57,  # la masse du ressort
    "file": [
        "data_john_4_case_2.csv",
        "data_john_5_case_2.csv",
        "data_john_7_case_2.csv",
        "data_john_9_case_2.csv",
        "data_john_1_case_2.csv",
    ],
    "nb_epoch": 1000,  # epoch number
    "save_rate": 20,  # rate to save
    "dynamic_weights": True,
    "lr_weights": 1e-1,  # si dynamic weights
    "weight_data": 0.33,
    "weight_border": 0.33,
    "batch_size": 10000,  # for the pde
    "Re": 100,
    "lr_init": 0.001,
    "gamma_scheduler": 0.999,  # pour la lr
    "nb_exit": 3,
    "nb_entry_branch": 100,
    "nb_entry_trunk": 3,
    "trunk_width": 32,
    "trunk_depth": 10,
    "branch_width": 32,
    "branch_depth": 10,
    "nb_branches": 32,
    "n_data_test": 5000,
    "nb_points": 12 * 12,  # le nombre de points pris par axe par pas de temps
    "x_min": -0.1,
    "x_max": 0.1,
    "y_min": -0.06,
    "y_max": 0.06,
    "t_min": 6.5,
    'nb_period': 10,
    'nb_period_plot': 2,
    "nb_points_close_cylinder": 50,  # le nombre de points proches du cylindre
    "rayon_close_cylinder": 0.015,
    "nb_points_border": 25,  # le nombrede points sur la condition init
}

param_adim = {"V": 1.0, "L": 0.025, "rho": 1.2}

In [4]:
time_start_charge = time.time()
nb_simu = len(hyper_param["file"])
x_full, y_full, t_full, x_branch_full = [], [], [], []
u_full, v_full, p_full = [], [], []
x_norm_full, y_norm_full, t_norm_full, x_branch_norm_full = [], [], [], []
u_norm_full, v_norm_full, p_norm_full = [], [], []

f = 0.5 * (hyper_param["H"] / hyper_param["m"]) ** 0.5
w0 = 2 * torch.pi * f
time_tot = hyper_param['nb_period_plot'] / f  # la fréquence de l'écoulement
t_max = hyper_param['t_min'] + hyper_param['nb_period'] * time_tot
t_max_plot = hyper_param['t_min'] + time_tot

In [5]:
for k in range(nb_simu):
    df = pd.read_csv("data/" + hyper_param["file"][k])
    df_modified = df.loc[
        (df["Points:0"] >= hyper_param["x_min"])
        & (df["Points:0"] <= hyper_param["x_max"])
        & (df["Points:1"] >= hyper_param["y_min"])
        & (df["Points:1"] <= hyper_param["y_max"])
        & (df["Time"] > hyper_param["t_min"])
        & (df["Time"] < t_max)
        & (df["Points:2"] == 0.0)
        & (df["Points:0"] ** 2 + df["Points:1"] ** 2 > (0.025 / 2) ** 2),
        :,
    ].copy()

    # Adimensionnement
    x_full.append(
        torch.tensor(df_modified["Points:0"].to_numpy(), dtype=torch.float32)
        / param_adim["L"]
    )
    y_full.append(
        torch.tensor(df_modified["Points:1"].to_numpy(), dtype=torch.float32)
        / param_adim["L"]
    )
    time_without_modulo = df_modified["Time"].to_numpy() - hyper_param['t_min']
    time_with_modulo = hyper_param['t_min'] + time_without_modulo % (1/f)
    t_full.append(
        torch.tensor(time_with_modulo, dtype=torch.float32)
        / (param_adim["L"] / param_adim["V"])
    )
    x_branch_full.append(
            find_x_branch(
            hyper_param['ya0'][k],
            w0=w0,
            nb_branch=hyper_param['nb_entry_branch'],
            t_min=hyper_param['t_min'],
            t_max=t_max_plot).reshape(-1,1).repeat(1, x_full[-1].shape[0]).T
    )

    u_full.append(
        torch.tensor(df_modified["Velocity:0"].to_numpy(), dtype=torch.float32)
        / param_adim["V"]
    )
    v_full.append(
        torch.tensor(df_modified["Velocity:1"].to_numpy(), dtype=torch.float32)
        / param_adim["V"]
    )
    p_full.append(
        torch.tensor(df_modified["Pressure"].to_numpy(), dtype=torch.float32)
        / ((param_adim["V"] ** 2) * param_adim["rho"])
    )
    print(f"fichier n°{k} chargé")

# les valeurs pour renormaliser ou dénormaliser
mean_std = {
    "u_mean": torch.cat([u for u in u_full], dim=0).mean(),
    "v_mean": torch.cat([v for v in v_full], dim=0).mean(),
    "p_mean": torch.cat([p for p in p_full], dim=0).mean(),
    "x_mean": torch.cat([x for x in x_full], dim=0).mean(),
    "y_mean": torch.cat([y for y in y_full], dim=0).mean(),
    "t_mean": torch.cat([t for t in t_full], dim=0).mean(),
    "x_std": torch.cat([x for x in x_full], dim=0).std(),
    "y_std": torch.cat([y for y in y_full], dim=0).std(),
    "t_std": torch.cat([t for t in t_full], dim=0).std(),
    "u_std": torch.cat([u for u in u_full], dim=0).std(),
    "v_std": torch.cat([v for v in v_full], dim=0).std(),
    "p_std": torch.cat([p for p in p_full], dim=0).std(),
    "x_branch_std": torch.cat([x_b for x_b in x_branch_full], dim=0).std(dim=0),
    "x_branch_mean": torch.cat([x_b for x_b in x_branch_full], dim=0).mean(dim=0)
}

fichier n°0 chargé
fichier n°1 chargé
fichier n°2 chargé
fichier n°3 chargé
fichier n°4 chargé


In [6]:
X_trunk_full = torch.zeros((0, 3))
U_full = torch.zeros((0, 3))


In [7]:
X_trunk_full = torch.zeros((0, 3))
U_full = torch.zeros((0, 3))

for k in range(nb_simu):
    # Normalisation Z
    x_norm_full.append((x_full[k] - mean_std["x_mean"]) / mean_std["x_std"])
    y_norm_full.append((y_full[k] - mean_std["y_mean"]) / mean_std["y_std"])
    t_norm_full.append((t_full[k] - mean_std["t_mean"]) / mean_std["t_std"])
    p_norm_full.append((p_full[k] - mean_std["p_mean"]) / mean_std["p_std"])
    u_norm_full.append((u_full[k] - mean_std["u_mean"]) / mean_std["u_std"])
    v_norm_full.append((v_full[k] - mean_std["v_mean"]) / mean_std["v_std"])
    x_branch_norm_full.append((x_branch_full[k] - mean_std["x_branch_mean"]) / mean_std["x_branch_std"])
    X_trunk_full = torch.cat(
        (
            X_trunk_full,
            torch.stack(
                (
                    x_norm_full[-1],
                    y_norm_full[-1],
                    t_norm_full[-1],
                ),
                dim=1,
            ),
        )
    )
    U_full = torch.cat(
        (
            U_full,
            torch.stack((u_norm_full[-1], v_norm_full[-1], p_norm_full[-1]), dim=1),
        )
    )



In [8]:
X_branch_full = (torch.cat([x_b for x_b in x_branch_norm_full]))

In [9]:
X_branch_full = (torch.cat([x_b for x_b in x_branch_norm_full]))
X_trunk_train = torch.zeros((0, 3))
U_train = torch.zeros((0, 3))
X_branch_train = torch.zeros((0, hyper_param['nb_entry_branch']))
print("Starting X_train")
for nb in range(len(x_full)):
    print(f"Simu n°{nb}/{len(t_norm_full)}")
    print(f"Time:{(time.time()-time_start_charge):.3f}")
    for time_ in torch.unique(t_norm_full[nb]):
        # les points autour du cylindre dans un rayon de hyper_param['rayon_proche']
        masque = (
            ((x_full[nb] ** 2 + y_full[nb] ** 2) < ((hyper_param['rayon_close_cylinder'] / param_adim["L"]) ** 2))
            & (t_norm_full[nb] == time_)
        )
        indices = torch.randperm(len(x_norm_full[nb][masque]))[
            : hyper_param["nb_points_close_cylinder"]
        ]

        new_x = torch.stack(
            (
                x_norm_full[nb][masque][indices],
                y_norm_full[nb][masque][indices],
                t_norm_full[nb][masque][indices],
            ),
            dim=1,
        )
        new_y = torch.stack(
            (
                u_norm_full[nb][masque][indices],
                v_norm_full[nb][masque][indices],
                p_norm_full[nb][masque][indices],
            ),
            dim=1,
        )
        X_trunk_train = torch.cat((X_trunk_train, new_x))
        U_train = torch.cat((U_train, new_y))
        X_branch_train = torch.cat((X_branch_train, x_branch_norm_full[nb][indices, :]), dim=0)


        # Les points avec 'latin hypercube sampling'
        time_start_lhs = time.time()

        masque = (t_norm_full[nb] == time_) 
        if x_norm_full[nb][masque].size(0) > 0:
            indices = torch.randperm(x_norm_full[nb][masque].size(0))[
                : hyper_param["nb_points"]
            ]
            new_x = torch.stack(
                (
                    x_norm_full[nb][masque][indices],
                    y_norm_full[nb][masque][indices],
                    t_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            new_y = torch.stack(
                (
                    u_norm_full[nb][masque][indices],
                    v_norm_full[nb][masque][indices],
                    p_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            X_trunk_train = torch.cat((X_trunk_train, new_x))
            U_train = torch.cat((U_train, new_y))
            X_branch_train = torch.cat((X_branch_train, x_branch_norm_full[nb][indices, :]))
indices = torch.randperm(X_trunk_train.size(0))
X_trunk_train = X_trunk_train[indices]
U_train = U_train[indices]
X_branch_train = X_branch_train[indices]
print("X_train OK")

Starting X_train
Simu n°0/5
Time:5.050
Simu n°1/5
Time:6.214
Simu n°2/5
Time:7.696
Simu n°3/5
Time:9.391
Simu n°4/5
Time:12.084
X_train OK


In [12]:
# les points du bord
teta_int = torch.linspace(0, 2 * torch.pi, hyper_param["nb_points_border"])
X_trunk_border = torch.empty((0, 3))
X_branch_border = torch.empty((0, hyper_param['nb_entry_branch']))
x_ = (
    (((0.025 / 2) * torch.cos(teta_int)) / param_adim["L"]) - mean_std["x_mean"]
) / mean_std["x_std"]
y_ = (
    (((0.025 / 2) * torch.sin(teta_int)) / param_adim["L"]) - mean_std["y_mean"]
) / mean_std["y_std"]

for nb in range(len(x_branch_full)):
    for time_ in torch.unique(t_norm_full[nb]):
        new_x = torch.stack(
            (x_, y_, torch.ones_like(x_) * time_), dim=1
        )
        X_trunk_border = torch.cat((X_trunk_border, new_x))
        X_branch_border = torch.cat((X_branch_border, x_branch_norm_full[nb][0].reshape(-1, 1).T.repeat(x_.shape[0], 1)))
    indices = torch.randperm(X_trunk_border.size(0))
    X_trunk_border = X_trunk_border[indices]
    X_branch_border = X_branch_border[indices]
print("X_border OK")

teta_int_test = torch.linspace(0, 2 * torch.pi, 15)
X_trunk_border_test = torch.zeros((0, 3))
X_branch_border_test = torch.empty((0, hyper_param['nb_entry_branch']))
x_ = (
    (((0.025 / 2) * torch.cos(teta_int_test)) / param_adim["L"])
    - mean_std["x_mean"]
) / mean_std["x_std"]
y_ = (
    (((0.025 / 2) * torch.sin(teta_int_test)) / param_adim["L"])
    - mean_std["y_mean"]
) / mean_std["y_std"]

for nb in range(len(x_branch_full)):
    for time_ in torch.unique(t_norm_full[nb]):
        new_x = torch.stack(
            (x_, y_, torch.ones_like(x_) * time_), dim=1
        )
        X_trunk_border_test = torch.cat((X_trunk_border_test, new_x))
        X_branch_border_test = torch.cat((X_branch_border_test, x_branch_norm_full[nb][0].reshape(-1, 1).T.repeat(x_.shape[0], 1)))

points_coloc_test = np.random.choice(
    len(X_trunk_full), hyper_param["n_data_test"], replace=False
)
X_trunk_test_data = X_trunk_full[points_coloc_test]
U_test_data = U_full[points_coloc_test]
X_branch_test_data = X_branch_full[points_coloc_test]

X_border OK


In [13]:
X_test_data = (X_branch_test_data, X_trunk_test_data)
X_train = (X_branch_train, X_trunk_train)
X_border = (X_branch_border, X_trunk_border)
X_border_test = (X_branch_border_test, X_trunk_border_test)
X_full = (X_branch_full, X_trunk_full)


In [ ]:
return X_train, U_train, X_full, U_full, X_border, X_border_test, mean_std, X_test_data, U_test_data

In [43]:
X_test_data.shape

torch.Size([5000, 3])

In [44]:
X_test_data_branch.shape

torch.Size([5000, 100])

In [51]:
X_full_branch[0, :]

tensor([ 2.2807,  2.2466,  2.2230,  2.2072,  2.1977,  2.1938,  2.1951,  2.2017,
         2.2142,  2.2336,  2.2620,  2.3028,  2.3620,  2.4509,  2.5935,  2.8507,
         3.4331,  5.8978, -8.6166, -3.7163, -2.9510, -2.6437, -2.4804, -2.3810,
        -2.3157, -2.2710, -2.2398, -2.2183, -2.2043, -2.1963, -2.1937, -2.1962,
        -2.2042, -2.2183, -2.2398, -2.2709, -2.3156, -2.3808, -2.4801, -2.6431,
        -2.9498, -3.7126, -8.5659,  5.9153,  3.4357,  2.8517,  2.5940,  2.4512,
         2.3622,  2.3030,  2.2621,  2.2337,  2.2142,  2.2017,  2.1951,  2.1938,
         2.1977,  2.2071,  2.2229,  2.2465,  2.2806,  2.3296,  2.4017,  2.5130,
         2.7009,  3.0726,  4.1171, 21.8137, -4.7618, -3.2305, -2.7699, -2.5510,
        -2.4252, -2.3452, -2.2913, -2.2540, -2.2280, -2.2105, -2.1996, -2.1943,
        -2.1943, -2.1995, -2.2104, -2.2280, -2.2539, -2.2912, -2.3450, -2.4249,
        -2.5506, -2.7691, -3.2286, -4.7531, 22.2451,  4.1221,  3.0741,  2.7016,
         2.5135,  2.4019,  2.3297,  2.28

In [46]:
x_.shape[0]

15

In [47]:
x_branch_norm_full[nb][0].reshape(-1, 1).T.repeat(x_.shape[0], 1)

tensor([[-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
        [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
        [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
        ...,
        [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
        [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494],
        [-1.2494, -1.2494, -1.2494,  ..., -1.2494, -1.2494, -1.2494]])

In [48]:


def charge_data(hyper_param, param_adim):
    """
    Charge the data of X_full, U_full with every points
    And X_train, U_train with less points
    """
    # La data
    # On adimensionne la data

    time_start_charge = time.time()
    nb_simu = len(hyper_param["file"])
    x_full, y_full, t_full, x_branch = [], [], [], []
    u_full, v_full, p_full = [], [], []
    x_norm_full, y_norm_full, t_norm_full, x_branch_full = [], [], [], []
    u_norm_full, v_norm_full, p_norm_full = [], [], []

    f = 0.5 * (hyper_param["H"] / hyper_param["m"]) ** 0.5
    w0 = 2 * torch.pi * f
    time_tot = hyper_param['nb_period_plot'] / f  # la fréquence de l'écoulement
    t_max = hyper_param['t_min'] + hyper_param['nb_period'] * time_tot
    for k in range(nb_simu):
        df = pd.read_csv("data/" + hyper_param["file"][k])
        df_modified = df.loc[
            (df["Points:0"] >= hyper_param["x_min"])
            & (df["Points:0"] <= hyper_param["x_max"])
            & (df["Points:1"] >= hyper_param["y_min"])
            & (df["Points:1"] <= hyper_param["y_max"])
            & (df["Time"] > hyper_param["t_min"])
            & (df["Time"] < t_max)
            & (df["Points:2"] == 0.0)
            & (df["Points:0"] ** 2 + df["Points:1"] ** 2 > (0.025 / 2) ** 2),
            :,
        ].copy()
        #hyper_param["ya0"][k]

        # Adimensionnement
        x_full.append(
            torch.tensor(df_modified["Points:0"].to_numpy(), dtype=torch.float32)
            / param_adim["L"]
        )
        y_full.append(
            torch.tensor(df_modified["Points:1"].to_numpy(), dtype=torch.float32)
            / param_adim["L"]
        )
        time_without_modulo = df_modified["Time"].to_numpy() - hyper_param['t_min']
        time_with_modulo = hyper_param['t_min'] + time_without_modulo % (1/f)
        t_full.append(
            torch.tensor(time_with_modulo, dtype=torch.float32)
            / (param_adim["L"] / param_adim["V"])
        )
        x_branch_full.append(
            find_x_branch(
                hyper_param['ya0'][k],
                w0=w0,
                nb_branch=hyper_param['nb_entry_branch'],
                t_min=hyper_param['t_min'],
                t_max=t_max)
        )

        u_full.append(
            torch.tensor(df_modified["Velocity:0"].to_numpy(), dtype=torch.float32)
            / param_adim["V"]
        )
        v_full.append(
            torch.tensor(df_modified["Velocity:1"].to_numpy(), dtype=torch.float32)
            / param_adim["V"]
        )
        p_full.append(
            torch.tensor(df_modified["Pressure"].to_numpy(), dtype=torch.float32)
            / ((param_adim["V"] ** 2) * param_adim["rho"])
        )
        print(f"fichier n°{k} chargé")

    # les valeurs pour renormaliser ou dénormaliser
    mean_std = {
        "u_mean": torch.cat([u for u in u_full], dim=0).mean(),
        "v_mean": torch.cat([v for v in v_full], dim=0).mean(),
        "p_mean": torch.cat([p for p in p_full], dim=0).mean(),
        "x_mean": torch.cat([x for x in x_full], dim=0).mean(),
        "y_mean": torch.cat([y for y in y_full], dim=0).mean(),
        "t_mean": torch.cat([t for t in t_full], dim=0).mean(),
        "x_std": torch.cat([x for x in x_full], dim=0).std(),
        "y_std": torch.cat([y for y in y_full], dim=0).std(),
        "t_std": torch.cat([t for t in t_full], dim=0).std(),
        "u_std": torch.cat([u for u in u_full], dim=0).std(),
        "v_std": torch.cat([v for v in v_full], dim=0).std(),
        "p_std": torch.cat([p for p in p_full], dim=0).std(),
        "ya0_mean": torch.cat([ya0 for ya0 in ya0_full], dim=0).mean(),
        "ya0_std": torch.cat([ya0 for ya0 in ya0_full], dim=0).std(),
    }



    X_full = torch.zeros((0, 4))
    U_full = torch.zeros((0, 3))
    for k in range(nb_simu):
        # Normalisation Z
        x_norm_full.append((x_full[k] - mean_std["x_mean"]) / mean_std["x_std"])
        y_norm_full.append((y_full[k] - mean_std["y_mean"]) / mean_std["y_std"])
        t_norm_full.append((t_full[k] - mean_std["t_mean"]) / mean_std["t_std"])
        ya0_norm_full.append((ya0_full[k] - mean_std["ya0_mean"]) / mean_std["ya0_std"])
        p_norm_full.append((p_full[k] - mean_std["p_mean"]) / mean_std["p_std"])
        u_norm_full.append((u_full[k] - mean_std["u_mean"]) / mean_std["u_std"])
        v_norm_full.append((v_full[k] - mean_std["v_mean"]) / mean_std["v_std"])
        X_full = torch.cat(
            (
                X_full,
                torch.stack(
                    (
                        x_norm_full[-1],
                        y_norm_full[-1],
                        t_norm_full[-1],
                        ya0_norm_full[-1],
                    ),
                    dim=1,
                ),
            )
        )
        U_full = torch.cat(
            (
                U_full,
                torch.stack((u_norm_full[-1], v_norm_full[-1], p_norm_full[-1]), dim=1),
            )
        )


    X_train = torch.zeros((0, 4))
    U_train = torch.zeros((0, 3))
    print("Starting X_train")
    for nb, ya0_ in enumerate(hyper_param["ya0"]):
        print(f"Simu n°{nb}/{len(hyper_param['ya0'])}")
        print(f"Time:{(time.time()-time_start_charge):.3f}")
        for time_ in torch.unique(t_norm_full[nb]):
            # les points autour du cylindre dans un rayon de hyper_param['rayon_proche']
            masque = (
                ((x_full[nb] ** 2 + y_full[nb] ** 2) < ((hyper_param['rayon_close_cylinder'] / param_adim["L"]) ** 2))
                & (t_norm_full[nb] == time_)
                # & (ya0_norm_full[nb] == ya0_)
            )
            indices = torch.randperm(len(x_norm_full[nb][masque]))[
                : hyper_param["nb_points_close_cylinder"]
            ]

            new_x = torch.stack(
                (
                    x_norm_full[nb][masque][indices],
                    y_norm_full[nb][masque][indices],
                    t_norm_full[nb][masque][indices],
                    ya0_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            new_y = torch.stack(
                (
                    u_norm_full[nb][masque][indices],
                    v_norm_full[nb][masque][indices],
                    p_norm_full[nb][masque][indices],
                ),
                dim=1,
            )
            X_train = torch.cat((X_train, new_x))
            U_train = torch.cat((U_train, new_y))


            # Les points avec 'latin hypercube sampling'
            time_start_lhs = time.time()

            masque = (t_norm_full[nb] == time_) 
            if x_norm_full[nb][masque].size(0) > 0:
                indices = torch.randperm(x_norm_full[nb][masque].size(0))[
                    : hyper_param["nb_points"]
                ]
                new_x = torch.stack(
                    (
                        x_norm_full[nb][masque][indices],
                        y_norm_full[nb][masque][indices],
                        t_norm_full[nb][masque][indices],
                        ya0_norm_full[nb][masque][indices],
                    ),
                    dim=1,
                )
                new_y = torch.stack(
                    (
                        u_norm_full[nb][masque][indices],
                        v_norm_full[nb][masque][indices],
                        p_norm_full[nb][masque][indices],
                    ),
                    dim=1,
                )
                X_train = torch.cat((X_train, new_x))
                U_train = torch.cat((U_train, new_y))
    indices = torch.randperm(X_train.size(0))
    X_train = X_train[indices]
    U_train = U_train[indices]
    print("X_train OK")

    # les points du bord
    teta_int = torch.linspace(0, 2 * torch.pi, hyper_param["nb_points_border"])
    X_border = torch.empty((0, 4))
    x_ = (
        (((0.025 / 2) * torch.cos(teta_int)) / param_adim["L"]) - mean_std["x_mean"]
    ) / mean_std["x_std"]
    y_ = (
        (((0.025 / 2) * torch.sin(teta_int)) / param_adim["L"]) - mean_std["y_mean"]
    ) / mean_std["y_std"]

    for nb, ya0_ in enumerate(X_train[:, 3].unique()):
        for time_ in torch.unique(t_norm_full[nb]):
            new_x = torch.stack(
                (x_, y_, torch.ones_like(x_) * time_, torch.ones_like(x_) * ya0_), dim=1
            )
            X_border = torch.cat((X_border, new_x))
        indices = torch.randperm(X_border.size(0))
        X_border = X_border[indices]
    print("X_border OK")

    teta_int_test = torch.linspace(0, 2 * torch.pi, 15)
    X_border_test = torch.zeros((0, 4))
    x_ = (
        (((0.025 / 2) * torch.cos(teta_int_test)) / param_adim["L"])
        - mean_std["x_mean"]
    ) / mean_std["x_std"]
    y_ = (
        (((0.025 / 2) * torch.sin(teta_int_test)) / param_adim["L"])
        - mean_std["y_mean"]
    ) / mean_std["y_std"]

    for nb, ya0_ in enumerate(X_train[:, 3].unique()):
        for time_ in torch.unique(t_norm_full[nb]):
            new_x = torch.stack(
                (x_, y_, torch.ones_like(x_) * time_, torch.ones_like(x_) * ya0_), dim=1
            )
            X_border_test = torch.cat((X_border_test, new_x))

    points_coloc_test = np.random.choice(
        len(X_full), hyper_param["n_data_test"], replace=False
    )
    X_test_data = X_full[points_coloc_test]
    U_test_data = U_full[points_coloc_test]
    return X_train, U_train, X_full, U_full, X_border, X_border_test, mean_std, X_test_data, U_test_data

